In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
sns.set(style="whitegrid")
import os
import pandas as pd

# Dossier contenant les fichiers clean
data_dir = "data/clean"  # adapte si ton dossier diffère

# Chargement automatique de toutes les bases *_clean.csv
dfs = {}
for file in os.listdir(data_dir):
    if file.endswith("_clean.csv"):
        name = file.replace(".csv", "")
        dfs[name] = pd.read_csv(os.path.join(data_dir, file))
        print(f"✅ Chargé : {name} ({len(dfs[name])} lignes)")

# Charger la table de référence si elle existe
if "communes_ref_clean" in dfs:
    communes_ref = dfs["communes_ref_clean"]
    print(f"\n📍 communes_ref prête : {len(communes_ref)} communes.")
else:
    print("⚠️ communes_ref non trouvée dans les fichiers clean.")

print("✅ Librairies importées avec succès.")

NameError: name 'false' is not defined

In [2]:
# 2. Chargement des bases de données nettoyées
data_folder = "data/clean"
clean_files = [f for f in os.listdir(data_folder) if f.endswith(".csv")]

dfs = {}
for f in clean_files:
    path = os.path.join(data_folder, f)
    name = f.replace(".csv", "")
    try:
        df = pd.read_csv(path, encoding="utf-8", sep=",", low_memory=False)
    except UnicodeDecodeError:
        df = pd.read_csv(path, encoding="latin-1", sep=";", low_memory=False)
    dfs[name] = df

for name, df in dfs.items():
    print(f"\n{name.upper()} : {df.shape[0]} lignes × {df.shape[1]} colonnes")
    print(list(df.columns[:8]))
    display(df.head(3))

possible_keys = ["code_commune", "code_insee", "Code_commune", "nom_commune", "Commune", "ville"]
for name, df in dfs.items():
    found = [col for col in df.columns if col in possible_keys]
    print(f"{name}: {found if found else 'aucune clé de jointure trouvée'}")


NameError: name 'os' is not defined

In [3]:
# Étape 2 — Harmonisation des clés

# Normalisation des clés de jointure
for name, df in dfs.items():
    if "code_commune" not in df.columns:
        for alt in ["Code_commune", "code_insee", "Commune", "nom_commune", "ville"]:
            if alt in df.columns:
                df.rename(columns={alt: "code_commune"}, inplace=True)
                break
    if "code_commune" in df.columns:
        df["code_commune"] = df["code_commune"].astype(str).str.zfill(5)
        dfs[name] = df

# Vérification de la présence et de la cohérence de la clé
for name, df in dfs.items():
    if "code_commune" in df.columns:
        print(f"\n{name.upper()} : clé détectée")
        print(f" - Type : {df['code_commune'].dtype}")
        print(f" - Nb valeurs uniques : {df['code_commune'].nunique()}")
        print(f" - Nb lignes : {len(df)}")
        print(f" - Doublons potentiels : {len(df) - df['code_commune'].nunique()}")
        display(df[['code_commune']].head(5))
    else:
        print(f"\n⚠️ {name.upper()} : aucune clé 'code_commune' détectée")


NameError: name 'dfs' is not defined

In [ ]:
import re, itertools, unicodedata
import pandas as pd
import numpy as np

def _strip_accents(s):
    if not isinstance(s, str): return s
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def norm_insee(x):
    if pd.isna(x): return None
    s = str(x).strip()
    s = re.sub(r"\.0$", "", s)
    s = re.sub(r"\D", "", s)
    if not re.fullmatch(r"\d{5}", s): return None
    return s

def norm_postal(x):
    if pd.isna(x): return None
    s = re.sub(r"\D", "", str(x).strip())
    if not re.fullmatch(r"\d{5}", s): return None
    return s

def norm_commune(x):
    if pd.isna(x): return None
    s = _strip_accents(str(x).upper().strip())
    s = re.sub(r"[\s'\-]", "", s)
    return s if s and not s.isdigit() else None

def norm_year(x):
    if pd.isna(x): return None
    m = re.search(r"\b(19\d{2}|20\d{2})\b", str(x))
    if not m: return None
    y = int(m.group(1))
    return y if 1900 <= y <= 2030 else None

def norm_lat(x):
    try: return round(float(x), 3)
    except: return None

def norm_lon(x):
    try: return round(float(x), 3)
    except: return None

# ignorer explicitement ces colonnes
IGNORE_NAME_PAT = re.compile(r"(?:^|_)(id|identifiant|mutation|numero|uuid|hash)(?:_|$)", re.I)

def guess_type(colname, s):
    name = colname.lower()

    if IGNORE_NAME_PAT.search(name):
        return None

    if any(k in name for k in ["insee","code_commune","codeinsee"]):
        return "insee"
    if any(k in name for k in ["cp","code_postal","postal"]):
        return "postal"
    if any(k in name for k in ["commune","ville","nom_ville","nomcommune"]):
        return "commune"
    if any(k in name for k in ["annee","year","date"]):
        return "year"
    if any(k in name for k in ["lat","latitude"]):
        return "lat"
    if any(k in name for k in ["lon","lng","longitude"]):
        return "lon"
    return None

def apply_norm(typ, series):
    if typ == "insee":   return series.map(norm_insee)
    if typ == "postal":  return series.map(norm_postal)
    if typ == "commune": return series.map(norm_commune)
    if typ == "year":    return series.map(norm_year)
    if typ == "lat":     return series.map(norm_lat)
    if typ == "lon":     return series.map(norm_lon)
    return None

profiles = {}
MIN_UNIQUES = 20

for dname, df in dfs.items():
    prof = {}
    for col in df.columns:
        typ = guess_type(col, df[col])
        if typ is None: 
            continue
        normed = apply_norm(typ, df[col])
        if normed is None:
            continue
        vals = pd.Series(normed).dropna().unique()
        nuniq = len(vals)
        if nuniq >= MIN_UNIQUES:
            prof[col] = (typ, normed, set(vals), nuniq)
    if prof:
        profiles[dname] = prof

def jaccard(a, b):
    inter = a & b
    uni = a | b
    return (len(inter) / len(uni) if len(uni) else 0.0, len(inter))

rows = []
for (d1, p1), (d2, p2) in itertools.combinations(profiles.items(), 2):
    for c1, (t1, s1, set1, n1) in p1.items():
        for c2, (t2, s2, set2, n2) in p2.items():
            same_type = (t1 == t2)
            allowed_mix = ({t1,t2} in [ {"commune","insee"}, {"commune","postal"} ])
            if not (same_type or allowed_mix):
                continue
            jac, inter = jaccard(set1, set2)
            cov1 = inter / n1 if n1 else 0
            cov2 = inter / n2 if n2 else 0

            if inter >= 50 or (jac >= 0.05 and cov1 >= 0.1 and cov2 >= 0.1):
                score = 0.6*jac + 0.2*cov1 + 0.2*cov2
                rows.append({
                    "dataset_left": d1, "col_left": c1, "type_left": t1, "unique_left": n1,
                    "dataset_right": d2, "col_right": c2, "type_right": t2, "unique_right": n2,
                    "intersection": inter, "jaccard": round(jac,4),
                    "cov_left": round(cov1,4), "cov_right": round(cov2,4),
                    "match_score": round(score,4)
                })

report = pd.DataFrame(rows).sort_values(["match_score","intersection"], ascending=[False, False]).head(50)
report.reset_index(drop=True, inplace=True)
report


In [ ]:
# === Pipeline Intégration & Analyse — version avec checks de progression ===
import re, unicodedata, pandas as pd, numpy as np

print("▶️ Démarrage pipeline...")

def strip_accents(s):
    if not isinstance(s, str): return s
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def norm_commune_name(s):
    if pd.isna(s): return None
    s = strip_accents(str(s).upper().strip())
    s = s.replace("’","'").replace("-", " ")
    s = re.sub(r"[\s'-]", "", s)
    s = re.sub(r"\bPARIS\s?(\d{1})\b", r"PARIS0\1", s)
    s = re.sub(r"\bPARIS\s?(\d{2})\b", r"PARIS\1", s)
    return s or None

def clean_insee(x):
    if pd.isna(x): return None
    s = str(x).strip()
    s = re.sub(r"\.0$", "", s)
    s = re.sub(r"\D", "", s)
    return s if re.fullmatch(r"\d{5}", s) else None

def best_col(df, candidates):
    for c in candidates:
        if c in df.columns: return c
    return None

def add_norm_name(df, name_cols=("nom_commune","Commune","commune","ville","LIBGEO")):
    col = best_col(df, name_cols)
    return df.assign(_nom_norm=df[col].map(norm_commune_name) if col else None)

def add_insee(df, insee_cols=("code_commune","code_insee","Code_commune","ninsee","codeinsee")):
    col = best_col(df, insee_cols)
    return df.assign(_insee=df[col].map(clean_insee) if col else None)

def coverage_pct(keys, ref_keys):
    if len(keys) == 0: return 0.0
    return round(keys.dropna().isin(ref_keys).mean()*100, 2)

success_steps = []

try:
    assert 'dfs' in globals() and isinstance(dfs, dict) and len(dfs) > 0, "Le dict 'dfs' doit être présent (bases chargées)."
    print(f"✅ dfs détecté ({len(dfs)} bases)")
    success_steps.append("dfs_ok")
except AssertionError as e:
    print("❌", e)
    raise


In [ ]:
# === 1) LIAISON — Création du pivot communes_ref (robuste) ===
import re, unicodedata
import pandas as pd

def _strip_acc(s):
    if not isinstance(s, str): return s
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def _norm_commune_name(s):
    if pd.isna(s): return None
    s = _strip_acc(str(s).upper().strip())
    s = s.replace("’","'").replace("-", " ")
    s = re.sub(r"[\s'-]", "", s)
    s = re.sub(r"\bPARIS\s?(\d{1})\b", r"PARIS0\1", s)
    s = re.sub(r"\bPARIS\s?(\d{2})\b", r"PARIS\1", s)
    return s or None

def _norm_insee_series(s):
    return (s.astype(str)
             .str.strip()
             .str.replace(r"\.0$","",regex=True)
             .str.replace(r"\D","",regex=True)
             .where(lambda x: x.str.fullmatch(r"\d{5}").fillna(False)))

def _build_communes_ref_from(df):
    code_col = next((c for c in ["code_commune","code_insee","Code_commune","ninsee","codeinsee"] if c in df.columns), None)
    name_col = next((c for c in ["nom_commune","Commune","commune","ville","LIBGEO"] if c in df.columns), None)
    out = pd.DataFrame()
    if code_col is not None:
        out["code_commune"] = _norm_insee_series(df[code_col])
    if name_col is not None:
        out["_nom_norm"] = df[name_col].map(_norm_commune_name)
    if "code_commune" not in out:
        out["code_commune"] = None
    out = out.dropna(subset=["code_commune"]).drop_duplicates("code_commune")
    return out

print("▶️ Étape 1 — Sélection du pivot")
print("Jeux dispo :", list(dfs.keys()))

cand_air   = [k for k in dfs if "air_parif_communes" in k.lower()]
cand_insee = [k for k in dfs if "insee" in k.lower()]
cand_dvf   = sorted([k for k in dfs if re.match(r"^dvf_\d{2}_clean$", k, flags=re.I)])

if cand_air:
    pivot_key = cand_air[0]
    print(f"✅ Pivot forcé (Airparif) : {pivot_key}")
    communes_ref = _build_communes_ref_from(dfs[pivot_key])
elif cand_insee:
    pivot_key = cand_insee[0]
    print(f"✅ Pivot INSEE : {pivot_key}")
    communes_ref = _build_communes_ref_from(dfs[pivot_key])
elif cand_dvf:
    print("✅ Pivot DVF = concat de tous les départements")
    tmp = pd.concat([dfs[k] for k in cand_dvf], ignore_index=True)
    communes_ref = _build_communes_ref_from(tmp)
else:
    raise ValueError("Aucun pivot exploitable trouvé parmi dfs.")

communes_ref = communes_ref.dropna(subset=["code_commune"]).drop_duplicates("code_commune")
nb_codes = communes_ref["code_commune"].nunique()
print(f"📌 communes_ref prête : {nb_codes} codes uniques")
display(communes_ref.head(10))

assert nb_codes > 0, "communes_ref vide."


In [ ]:
import re

dvf_keys = sorted([k for k in dfs if re.match(r"^dvf_\d{2}_clean$", k, flags=re.I)])
print("DVF trouvées :", dvf_keys)

for k in dvf_keys:
    df = dfs[k]
    print(f"\n— {k}")
    print("Colonnes dispo (extrait) :", [c for c in df.columns if c.lower() in 
          {"code_commune","code_insee","commune","nom_commune","ville","surface_reelle_bati","valeur_fonciere"}])
    for col in ["code_commune","code_insee","commune","nom_commune","ville"]:
        if col in df.columns:
            nnz = df[col].notna().sum()
            ex = df[col].dropna().astype(str).head(3).tolist()
            print(f"  {col}: {nnz} non-nuls, ex: {ex}")


In [ ]:
import pandas as pd, re, unicodedata

def strip_acc(s):
    if not isinstance(s,str): return s
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def norm_commune_name(s):
    if pd.isna(s): return None
    s = strip_acc(str(s).upper().strip())
    s = s.replace("’","'").replace("-", " ")
    s = re.sub(r"[\s'-]", "", s)
    s = re.sub(r"\bPARIS\s?(\d{1})\b", r"PARIS0\1", s)
    s = re.sub(r"\bPARIS\s?(\d{2})\b", r"PARIS\1", s)
    return s or None

def clean_insee(x):
    s = str(x).strip()
    s = re.sub(r"\.0$","", s)
    s = re.sub(r"\D","", s)
    return s if re.fullmatch(r"\d{5}", s) else None

# concat minimal DVF avec nom+code quand présent
pairs = []
for k in dvf_keys:
    df = dfs[k].copy()
    # choix d'un champ nom dispo
    name_col = next((c for c in ["nom_commune","commune","ville","Commune"] if c in df.columns), None)
    code_col = next((c for c in ["code_commune","code_insee","Code_commune"] if c in df.columns), None)
    if name_col is None or code_col is None: 
        continue
    tmp = df[[name_col, code_col]].dropna()
    if tmp.empty: 
        continue
    tmp["_nom_norm"] = tmp[name_col].map(norm_commune_name)
    tmp["code_commune"] = tmp[code_col].map(clean_insee)
    tmp = tmp.dropna(subset=["_nom_norm","code_commune"]).drop_duplicates(["code_commune","_nom_norm"])
    pairs.append(tmp[["_nom_norm","code_commune"]])

if pairs:
    dvf_name_map = pd.concat(pairs, ignore_index=True).drop_duplicates("code_commune")
    # merge pour enrichir communes_ref
    if "_nom_norm" in communes_ref.columns:
        communes_ref.drop(columns=["_nom_norm"], inplace=True)
    communes_ref = communes_ref.merge(dvf_name_map, on="code_commune", how="left")
    print(f"📝 communes_ref enrichi en noms via DVF — couverture noms: {communes_ref['_nom_norm'].notna().mean()*100:.1f}%")
else:
    print("ℹ️ Aucun couple nom+code exploitable trouvé dans DVF pour enrichir le pivot.")


In [ ]:
# === Enrichir communes_ref avec des noms depuis Éducation puis Transports ===
import re, unicodedata, pandas as pd

def _strip_acc(s):
    if not isinstance(s,str): return s
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def norm_commune_name(s):
    if pd.isna(s): return None
    s = _strip_acc(str(s).upper().strip())
    s = s.replace("’","'").replace("-", " ")
    s = re.sub(r"[\s'-]", "", s)
    s = re.sub(r"\bPARIS\s?(\d{1})\b", r"PARIS0\1", s)
    s = re.sub(r"\bPARIS\s?(\d{2})\b", r"PARIS\1", s)
    return s or None

def clean_insee(x):
    s = str(x).strip()
    s = re.sub(r"\.0$","", s)
    s = re.sub(r"\D","", s)
    return s if re.fullmatch(r"\d{5}", s) else None

def extract_nom_code(df):
    name_col = next((c for c in ["nom_commune","commune","ville","Commune","LIBGEO"] if c in df.columns), None)
    code_col = next((c for c in ["code_commune","code_insee","Code_commune","ninsee","codeinsee"] if c in df.columns), None)
    if not name_col or not code_col:
        return None
    tmp = df[[name_col, code_col]].dropna()
    if tmp.empty: return None
    tmp["_nom_norm"]   = tmp[name_col].map(norm_commune_name)
    tmp["code_commune"]= tmp[code_col].map(clean_insee)
    tmp = tmp.dropna(subset=["_nom_norm","code_commune"]).drop_duplicates("code_commune")
    return tmp[["_nom_norm","code_commune"]]

before = communes_ref["_nom_norm"].notna().mean()*100 if "_nom_norm" in communes_ref.columns else 0.0

edu_keys = [k for k in dfs if "annuaire_education" in k.lower() or "etablissement" in k.lower()]
if edu_keys:
    edu_map = extract_nom_code(dfs[edu_keys[0]])
    if edu_map is not None:
        communes_ref = communes_ref.merge(edu_map, on="code_commune", how="left") if "_nom_norm" not in communes_ref.columns \
                       else communes_ref.merge(edu_map.rename(columns={"_nom_norm":"_nom_norm_edu"}), on="code_commune", how="left")
        if "_nom_norm_edu" in communes_ref.columns:
            communes_ref["_nom_norm"] = communes_ref["_nom_norm"].fillna(communes_ref["_nom_norm_edu"])
            communes_ref.drop(columns=["_nom_norm_edu"], inplace=True)

tr_keys = [k for k in dfs if "transport" in k.lower() or "idf" in k.lower()]
if tr_keys:
    tr_map = extract_nom_code(dfs[tr_keys[0]])
    if tr_map is not None:
        communes_ref = communes_ref.merge(tr_map.rename(columns={"_nom_norm":"_nom_norm_tr"}), on="code_commune", how="left")
        communes_ref["_nom_norm"] = communes_ref["_nom_norm"].fillna(communes_ref["_nom_norm_tr"])
        communes_ref.drop(columns=["_nom_norm_tr"], inplace=True)

after = communes_ref["_nom_norm"].notna().mean()*100 if "_nom_norm" in communes_ref.columns else 0.0
print(f"📝 Couverture noms dans communes_ref : {before:.1f}% → {after:.1f}%")
display(communes_ref.head(8))


In [ ]:
# ============================================================
# ÉTAPE 2 — FUSION COMPLÈTE & LIAISON MULTI-BDD
# Version finale avec matching flou (similarité)
# ============================================================
import pandas as pd, re, numpy as np, unicodedata
from difflib import get_close_matches

# ============================================================
# 🔧 FONCTIONS UTILITAIRES
# ============================================================
def strip_acc(s):
    if not isinstance(s, str): return s
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def norm_commune_name(s):
    if pd.isna(s): return None
    s = strip_acc(str(s).upper().strip())
    s = s.replace("’","'").replace("-", " ")
    s = re.sub(r"[\s'-]", "", s)
    s = re.sub(r"\bPARIS\s?(\d{1})\b", r"PARIS0\1", s)
    s = re.sub(r"\bPARIS\s?(\d{2})\b", r"PARIS\1", s)
    return s or None

def clean_insee(x):
    s = str(x).strip()
    s = re.sub(r"\.0$", "", s)
    s = re.sub(r"\D", "", s)
    return s if re.fullmatch(r"\d{5}", s) else None

def coverage_pct(keys, ref):
    if len(keys) == 0: return 0.0
    return round(keys.dropna().isin(ref).mean()*100, 2)

def safe_num_cols(df, exclude):
    return [c for c in df.columns if c not in exclude and df[c].dtype.kind in "biufc"]

def build_nom2code(communes_ref):
    """Mapping nom_norm -> code_commune avec index unique"""
    df = communes_ref.dropna(subset=["_nom_norm","code_commune"]).copy()
    df = df.sort_values("code_commune").drop_duplicates("_nom_norm")
    return df.set_index("_nom_norm")["code_commune"]

def enrich_codes_by_similarity(df, communes_ref, nom_col="nom_commune", seuil=0.8):
    """Matching flou pour trouver un code_commune via similarité"""
    if nom_col not in df.columns: 
        return df
    noms_ref = list(communes_ref["_nom_norm"].dropna().unique())
    map_nom2code = communes_ref.set_index("_nom_norm")["code_commune"].to_dict()

    df["_nom_norm"] = df[nom_col].map(norm_commune_name)
    df["code_commune_fuzzy"] = None

    for i, nom in enumerate(df["_nom_norm"]):
        if pd.isna(nom): 
            continue
        match = get_close_matches(nom, noms_ref, n=1, cutoff=seuil)
        if match:
            df.at[i, "code_commune_fuzzy"] = map_nom2code.get(match[0])
    return df

# ============================================================
# 🔗 INITIALISATION
# ============================================================
ref_codes = set(communes_ref["code_commune"])
print(f"📌 communes_ref prête : {len(ref_codes)} codes uniques | "
      f"{round(communes_ref['_nom_norm'].notna().mean()*100,1)}% noms renseignés.\n")

# ============================================================
# 1️⃣ DVF — Agrégation robuste
# ============================================================
dvf_keys = sorted([k for k in dfs if re.match(r"^dvf_\d{2}_clean$", k, flags=re.I)])
dvf_parts = []
for k in dvf_keys:
    df = dfs[k].copy()
    base_cols = [c for c in ["valeur_fonciere","surface_reelle_bati"] if c in df.columns]
    code_cols = [c for c in ["code_commune","code_insee","Code_commune"] if c in df.columns]
    name_cols = [c for c in ["nom_commune","commune","ville","Commune"] if c in df.columns]
    use_cols = list(dict.fromkeys(base_cols + code_cols + name_cols))
    dvf_parts.append(df[use_cols])

dvf_all = pd.concat(dvf_parts, ignore_index=True)
dvf_all["code_commune"] = None
for c in ["code_commune","code_insee","Code_commune"]:
    if c in dvf_all.columns:
        dvf_all["code_commune"] = dvf_all["code_commune"].fillna(dvf_all[c].map(clean_insee))

name_col = next((c for c in ["nom_commune","commune","ville","Commune"] if c in dvf_all.columns), None)
if name_col and "_nom_norm" in communes_ref.columns:
    dvf_all["_nom_norm"] = dvf_all[name_col].map(norm_commune_name)
    map_nom2code = build_nom2code(communes_ref)
    dvf_all.loc[dvf_all["code_commune"].isna(), "code_commune"] = dvf_all["_nom_norm"].map(map_nom2code)

for c in ["valeur_fonciere","surface_reelle_bati"]:
    if c in dvf_all.columns:
        dvf_all[c] = pd.to_numeric(dvf_all[c], errors="coerce")

dvf_ok = dvf_all.dropna(subset=["code_commune","valeur_fonciere","surface_reelle_bati"])
dvf_ok = dvf_ok[
    (dvf_ok["surface_reelle_bati"].between(8,400)) &
    (dvf_ok["valeur_fonciere"].between(10_000,10_000_000))
]
agg_dvf = (dvf_ok.groupby("code_commune", as_index=False)
           .agg(somme_valeur=("valeur_fonciere","sum"),
                somme_surface=("surface_reelle_bati","sum"),
                n_ventes=("surface_reelle_bati","size")))
agg_dvf["prix_m2"] = agg_dvf["somme_valeur"] / agg_dvf["somme_surface"]
print(f"✅ DVF agrégé : {agg_dvf.shape} | prix_m2 non-nuls: {round(agg_dvf['prix_m2'].notna().mean()*100,2)}%")
base = agg_dvf.copy()

# ============================================================
# 2️⃣ AIRPARIF — Pollution
# ============================================================
air_key = [k for k in dfs if "air_parif_communes" in k.lower()]
if air_key:
    air = dfs[air_key[0]].copy()
    code_col = next((c for c in ["code_commune","code_insee","ninsee","codeinsee"] if c in air.columns), None)
    air["code_commune"] = air[code_col].map(clean_insee)
    num_cols = safe_num_cols(air, {"code_commune"})
    air_agg = air.dropna(subset=["code_commune"]).groupby("code_commune", as_index=False)[num_cols].mean()
    base = base.merge(air_agg, on="code_commune", how="left")
    print("✅ base + Airparif :", base.shape)

# ============================================================
# 3️⃣ DPE — Énergie
# ============================================================
dpe_key = [k for k in dfs if "dpe" in k.lower()]
if dpe_key:
    dpe = dfs[dpe_key[0]].copy()
    code_col = next((c for c in ["code_commune","code_insee","Code_commune","ninsee","codeinsee"] if c in dpe.columns), None)
    name_col = next((c for c in ["nom_commune","commune","ville","Commune"] if c in dpe.columns), None)
    dpe["code_commune"] = dpe[code_col].map(clean_insee) if code_col else None
    if dpe["code_commune"].isna().any() and name_col and "_nom_norm" in communes_ref.columns:
        dpe["_nom_norm"] = dpe[name_col].map(norm_commune_name)
        map_nom2code = build_nom2code(communes_ref)
        dpe.loc[dpe["code_commune"].isna(), "code_commune"] = dpe["_nom_norm"].map(map_nom2code)
    class_cols = [c for c in dpe.columns if "classe" in c.lower()]
    if class_cols:
        cc = class_cols[0]
        dpe["_is_FG"] = dpe[cc].astype(str).str.upper().str.strip().isin(["F","G"])
        dpe_agg = (dpe.dropna(subset=["code_commune"])
                   .groupby("code_commune", as_index=False)
                   .agg(nb_dpe=("code_commune","size"),
                        part_FG=("_is_FG","mean")))
        base = base.merge(dpe_agg, on="code_commune", how="left")
        print("✅ base + DPE :", base.shape)

# ============================================================
# 4️⃣ ÉDUCATION
# ============================================================
edu_keys = [k for k in dfs if "annuaire_education" in k.lower() or "etablissement" in k.lower()]
if edu_keys:
    edu = dfs[edu_keys[0]].copy()
    name_col = next((c for c in ["nom_commune","commune","ville","Commune"] if c in edu.columns), None)
    if name_col and "_nom_norm" in communes_ref.columns:
        edu["_nom_norm"] = edu[name_col].map(norm_commune_name)
        map_nom2code = build_nom2code(communes_ref)
        edu["code_commune"] = edu["_nom_norm"].map(map_nom2code)
        edu = enrich_codes_by_similarity(edu, communes_ref, nom_col=name_col)
        edu["code_commune"] = edu["code_commune"].fillna(edu["code_commune_fuzzy"])
        edu.drop(columns=["code_commune_fuzzy"], inplace=True)
        edu_agg = edu.dropna(subset=["code_commune"]).groupby("code_commune", as_index=False).size().rename(columns={"size":"nb_etabs"})
        base = base.merge(edu_agg, on="code_commune", how="left")
        print("✅ base + Éducation :", base.shape)

# ============================================================
# 5️⃣ TRANSPORTS
# ============================================================
tr_keys = [k for k in dfs if "transport" in k.lower() or "idf" in k.lower()]
if tr_keys:
    tr = dfs[tr_keys[0]].copy()
    name_col = next((c for c in ["nom_commune","commune","ville","Commune"] if c in tr.columns), None)
    if name_col and "_nom_norm" in communes_ref.columns:
        tr["_nom_norm"] = tr[name_col].map(norm_commune_name)
        map_nom2code = build_nom2code(communes_ref)
        tr["code_commune"] = tr["_nom_norm"].map(map_nom2code)
        tr = enrich_codes_by_similarity(tr, communes_ref, nom_col=name_col)
        tr["code_commune"] = tr["code_commune"].fillna(tr["code_commune_fuzzy"])
        tr.drop(columns=["code_commune_fuzzy"], inplace=True)
        if "mode" in tr.columns:
            weights = {"RER":3,"TRAIN":3,"METRO":2,"TRAM":1,"BUS":0.5}
            tr["_w"] = tr["mode"].astype(str).str.upper().map(weights).fillna(0.5)
            tr_agg = tr.groupby("code_commune", as_index=False)["_w"].sum().rename(columns={"_w":"indice_connectivite"})
        else:
            tr_agg = tr.groupby("code_commune", as_index=False).size().rename(columns={"size":"nb_arrets"})
        base = base.merge(tr_agg, on="code_commune", how="left")
        print("✅ base + Transports :", base.shape)

# ============================================================
# 6️⃣ DÉLINQUANCE
# ============================================================
del_keys = [k for k in dfs if "delin" in k.lower()]
if del_keys:
    crim = dfs[del_keys[0]].copy()
    code_col = next((c for c in ["code_commune","code_insee","Code_commune","ninsee","codeinsee"] if c in crim.columns), None)
    name_col = next((c for c in ["nom_commune","commune","ville","Commune"] if c in crim.columns), None)
    crim["code_commune"] = crim[code_col].map(clean_insee) if code_col else None
    if crim["code_commune"].isna().any() and name_col and "_nom_norm" in communes_ref.columns:
        crim["_nom_norm"] = crim[name_col].map(norm_commune_name)
        map_nom2code = build_nom2code(communes_ref)
        crim.loc[crim["code_commune"].isna(), "code_commune"] = crim["_nom_norm"].map(map_nom2code)
    crim = enrich_codes_by_similarity(crim, communes_ref, nom_col=name_col)

    # si la colonne floue existe, on fusionne les infos
    if "code_commune_fuzzy" in crim.columns:
        crim["code_commune"] = crim["code_commune"].fillna(crim["code_commune_fuzzy"])
        crim.drop(columns=["code_commune_fuzzy"], inplace=True)
    else:
        print("⚠️ Aucun enrichissement fuzzy pour Délinquance (colonne absente).")

    if {"faits_total","population"}.issubset(crim.columns):
        crim_agg = (crim.dropna(subset=["code_commune"])
                    .groupby("code_commune", as_index=False)
                    .agg(faits_total=("faits_total","sum"),
                         population=("population","max")))
        crim_agg["taux_criminalite_pour_1k"] = (crim_agg["faits_total"]/crim_agg["population"])*1000
        base = base.merge(crim_agg[["code_commune","taux_criminalite_pour_1k"]], on="code_commune", how="left")
        print("✅ base + Délinquance :", base.shape)

# ============================================================
# 7️⃣ EXPORT & BILAN FINAL
# ============================================================
drop_cols = [c for c in ["somme_valeur","somme_surface"] if c in base.columns]
if drop_cols:
    base.drop(columns=drop_cols, inplace=True)

print("\n===== BILAN FINAL =====")
print(f"Communes finales: {base['code_commune'].nunique()} | Lignes: {len(base)}")
for col in ["prix_m2","part_FG","zone_encadree","indice_connectivite","taux_criminalite_pour_1k"]:
    if col in base.columns:
        print(f"{col}: {round(base[col].notna().mean()*100,2)}% non-nuls")

base.to_csv("data/master_table.csv", index=False, encoding="utf-8")
print("✅ Export → data/master_table.csv")
display(base.head(10))


In [ ]:
# ============================================================
# ÉTAPE 3 — ANALYSE & INDICATEURS COMPOSITES (robuste)
# ============================================================
import numpy as np
import pandas as pd

print("▶️ Chargement du master…")
base = pd.read_csv("data/master_table.csv", dtype={"code_commune": str})
print(f"✅ {len(base)} lignes, {base['code_commune'].nunique()} communes\n")

# ---------- Helpers ----------
def to_numeric(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def scale_0_100_robust(series, higher_is_better=True):
    """Min–Max avec garde-fous:
       - <3 valeurs non-null -> NaN
       - série constante -> 50 pour tous (ou 100 si higher_is_better et valeur == 0 pour un "risque" type part_FG)
       - coupe aux percentiles 2–98 pour limiter les outliers
    """
    s = series.astype(float).copy()
    n = s.notna().sum()
    if n < 3:
        return pd.Series(np.nan, index=s.index)

    # clamp aux percentiles
    lo, hi = np.nanpercentile(s, [2, 98])
    s = s.clip(lo, hi)

    vmin, vmax = np.nanmin(s), np.nanmax(s)
    if not np.isfinite(vmin) or not np.isfinite(vmax):
        return pd.Series(np.nan, index=s.index)

    if np.isclose(vmin, vmax, atol=1e-12):
        # si tout identique -> 50 neutre
        # cas utile: part_FG = 0 partout => 100 (meilleur) quand higher_is_better=False
        if higher_is_better:
            return pd.Series(50.0, index=s.index)
        else:
            # valeur faible = mieux ⇒ tout 100 si constant
            return pd.Series(100.0, index=s.index)

    if higher_is_better:
        scaled = 100 * (s - vmin) / (vmax - vmin)
    else:
        scaled = 100 * (vmax - s) / (vmax - vmin)
    return scaled

def safe_col(df, name, default=np.nan):
    return df[name] if name in df.columns else pd.Series(default, index=df.index)

# ---------- Préparation ----------
num_cols = ["prix_m2","n_ventes","part_FG","no2","pm10","o3",
            "nb_etabs","indice_connectivite","nb_arrets",
            "taux_criminalite_pour_1k","population","nb_dpe"]
base = to_numeric(base, [c for c in num_cols if c in base.columns])

# Densité d'établissements si population dispo
if "population" in base.columns and base["population"].notna().any():
    base["etabs_par_10k"] = (safe_col(base, "nb_etabs", 0) / base["population"]) * 10_000
else:
    base["etabs_par_10k"] = np.nan

# ---------- Scores unitaires ----------
# Marché
base["score_prix"]      = scale_0_100_robust(safe_col(base, "prix_m2"), higher_is_better=False)  # moins cher = mieux
base["score_liquidite"] = scale_0_100_robust(safe_col(base, "n_ventes"), higher_is_better=True)
base["score_marche"]    = 0.6*base["score_prix"] + 0.4*base["score_liquidite"]

# Énergie
base["score_energie"] = 100 - scale_0_100_robust(safe_col(base, "part_FG"), higher_is_better=True)
if "nb_dpe" in base.columns:
    boost = scale_0_100_robust(base["nb_dpe"], higher_is_better=True) * 0.1  # +10% max si gros volume DPE
    base["score_energie"] = np.clip(base["score_energie"]*0.9 + boost, 0, 100)

# Pollution (plus bas = mieux)
poll_inputs = [c for c in ["no2","pm10","o3"] if c in base.columns]
if poll_inputs:
    base["score_pollution"] = scale_0_100_robust(base[poll_inputs].mean(axis=1), higher_is_better=False)
else:
    base["score_pollution"] = np.nan

# Éducation / attractivité
base["score_education"] = scale_0_100_robust(
    base["etabs_par_10k"].fillna(safe_col(base, "nb_etabs")), higher_is_better=True
)

# Transports
if "indice_connectivite" in base.columns and base["indice_connectivite"].notna().any():
    base["score_transport"] = scale_0_100_robust(base["indice_connectivite"], higher_is_better=True)
elif "nb_arrets" in base.columns and base["nb_arrets"].notna().any():
    base["score_transport"] = scale_0_100_robust(base["nb_arrets"], higher_is_better=True)
else:
    base["score_transport"] = np.nan

# Sécurité
if "taux_criminalite_pour_1k" in base.columns and base["taux_criminalite_pour_1k"].notna().any():
    base["score_securite"] = 100 - scale_0_100_robust(base["taux_criminalite_pour_1k"], higher_is_better=True)
else:
    base["score_securite"] = np.nan

# Encadrement (pénalité)
penality = (base["zone_encadree"].fillna(False).astype(bool).map({True: -10.0, False: 0.0})
            if "zone_encadree" in base.columns else 0.0)

# ---------- Score global (pondérations auto-normalisées sur ce qui existe) ----------
weights = {
    "score_marche":    0.35,
    "score_transport": 0.15,
    "score_energie":   0.15,
    "score_education": 0.10,
    "score_securite":  0.15,
    "score_pollution": 0.10,
}
present = {k:v for k,v in weights.items() if k in base.columns and base[k].notna().any()}
w_sum = sum(present.values()) or 1.0
present = {k: v/w_sum for k,v in present.items()}

base["score_global"] = 0.0
for k, w in present.items():
    # remplace NaN par médiane de la colonne pour ne pas annuler le score
    base["score_global"] = base["score_global"] + w * base[k].fillna(base[k].median())

base["score_global"] = np.clip(base["score_global"] + penality, 0, 100)

# ---------- Contrôles & exports ----------
scores_cols = ["score_marche","score_transport","score_energie","score_education",
               "score_securite","score_pollution","score_global"]

print("🔎 Couverture des scores (non-nuls) :")
for c in scores_cols:
    if c in base.columns:
        print(f" - {c}: {round(base[c].notna().mean()*100,1)}%")

top = base.sort_values("score_global", ascending=False).head(10)
bottom = base.sort_values("score_global", ascending=True).head(10)

print("\n🏆 TOP 10 (toutes communes disponibles)"); 
display(top[["code_commune","prix_m2","score_global"] + [c for c in scores_cols if c!="score_global"]])
print("\n⚠️ BOTTOM 10"); 
display(bottom[["code_commune","prix_m2","score_global"] + [c for c in scores_cols if c!="score_global"]])

out = "data/master_with_scores.csv"
base.to_csv(out, index=False, encoding="utf-8")
print(f"\n✅ Export enrichi → {out}")


In [ ]:
# ============================================================
# 🧹 Nettoyage mémoire sécurisé
# ============================================================
import gc

# Libérer toutes les variables sauf les modules
for name in list(globals().keys()):
    if not name.startswith("_") and name not in ["gc", "pd", "np"]:
        del globals()[name]

gc.collect()
print("✅ Mémoire RAM libérée avec succès (variables temporaires supprimées).")
